In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import GradientBoostingRegressor
import geopandas as gpd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.inspection import permutation_importance
from sklearn.cluster import KMeans
import datetime as dt
!pip install haversine
from haversine import haversine
seed = 17
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm

In [14]:
! kaggle competitions download -c playground-series-s3e20

 80% 39.0M/48.9M [00:00<00:00, 114MB/s] 
100% 48.9M/48.9M [00:00<00:00, 108MB/s]


In [15]:
! unzip -qq playground-series-s3e20.zip

In [16]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [69]:
def get_id(row):
    return int(''.join(filter(str.isdigit, str(row['latitude']))) + ''.join(filter(str.isdigit, str(row['longitude']))))

train['id'] = train[['latitude', 'longitude']].apply(lambda row: get_id(row), axis=1)
test['id'] = test[['latitude', 'longitude']].apply(lambda row: get_id(row), axis=1)
new_ids = {id_: new_id for new_id, id_ in enumerate(train['id'].unique())}
train['id'] = train['id'].map(new_ids)
test['id'] = test['id'].map(new_ids)

def get_month(row):
    date = dt.datetime.strptime(f'{row["year"]}-{row["week_no"]+1}-1', "%Y-%W-%w")
    return date.month

rwanda_center = (-1.9607, 29.9707)
park_biega = (-1.8866, 28.4518)
kirumba = (-0.5658, 29.1714)
massif = (-3.42, 28.592)
lake = (-2.0073, 31.6269)

def cluster_features(df, cluster_centers):
    for i, cc in enumerate(cluster_centers.values()):
        df[f'cluster_{i}'] = df.apply(lambda x: haversine((x['latitude'], x['longitude']), cc, unit='ft'), axis=1)
    return df

y = train['emission']

def preprocessing(df):

    cols_save = ['id', 'latitude', 'longitude', 'year', 'week_no', 'Ozone_solar_azimuth_angle']
    df = df[cols_save]

    good_col = 'Ozone_solar_azimuth_angle'
    df[good_col] = df.groupby(['id', 'year'])[good_col].ffill().bfill()
    df[f'{good_col}_lag_1'] = df.groupby(['id', 'year'])[good_col].shift(1).fillna(0)

    df['rot_15_x'] = (np.cos(np.radians(15)) * df['longitude']) + \
                     (np.sin(np.radians(15)) * df['latitude'])

    df['rot_15_y'] = (np.cos(np.radians(15)) * df['latitude']) + \
                     (np.sin(np.radians(15)) * df['longitude'])

    df['rot_30_x'] = (np.cos(np.radians(30)) * df['longitude']) + \
                     (np.sin(np.radians(30)) * df['latitude'])

    df['rot_30_y'] = (np.cos(np.radians(30)) * df['latitude']) + \
                     (np.sin(np.radians(30)) * df['longitude'])

    for col, coors in zip(
        ['dist_rwanda', 'dist_park', 'dist_kirumba', 'dist_massif', 'dist_lake'],
        [rwanda_center, park_biega, kirumba, massif, lake]
    ):
        df[col] = df.apply(lambda x: haversine((x['latitude'], x['longitude']), coors, unit='ft'), axis=1)

    df['month'] = df[['year', 'week_no']].apply(lambda row: get_month(row), axis=1)
    df['is_covid'] = (df['year'] == 2020) & (df['month'] > 2) | (df['year'] == 2021) & (df['month'] == 1)
    df['is_lockdown'] = (df['year'] == 2020) & ((df['month'].isin([3,4])))

    df.fillna(0, inplace=True)
    return df

train = preprocessing(train)
test = preprocessing(test)

df = pd.concat([train, test], axis=0, ignore_index=True)
coordinates = df[['latitude', 'longitude']].values
clustering = KMeans(n_clusters=12, max_iter=1000, random_state=seed).fit(coordinates)
cluster_centers = {i: tuple(centroid) for i, centroid in enumerate(clustering.cluster_centers_)}
df = cluster_features(df, cluster_centers)

train = df.iloc[:-len(test),:]
test = df.iloc[-len(test):,:]
del df

X = train.drop('id', axis=1)
test = test.drop('id', axis=1)
test=test.reset_index(drop=True)

In [60]:
def cross_validation(model,n_fold, n_splits, X, y):
  models = [ ]
  scores = [ ]
  for i in range(n_fold):
     kf = KFold(n_splits=n_splits, shuffle=True, random_state=None)

     for num, (train_index, test_index) in enumerate(kf.split(X)):
       X_train, X_test = X.iloc[train_index], X.iloc[test_index]
       y_train, y_test = y.iloc[train_index], y.iloc[test_index]

       model.fit(X_train, y_train)

       models.append(model)

       y_pred = model.predict(X_test)
       score = np.sqrt(mse(y_pred, y_test))
       scores.append(score)

       #print(f"fold: {num} rmse: {score}")
  print("total rmse", np.mean(scores, dtype="float16"), 'std',  np.std(scores).round(4))

  return models
def cross_pred(model, X):
  pred = pd.DataFrame()
  for n, est in enumerate(model):
    pred[n]= est.predict(X)
  pred['mean'] = pred.mean(axis = 1)
  return pred['mean']

In [35]:
rfr = cross_validation(RandomForestRegressor(n_estimators = 10, max_depth = 400, verbose = 0), 3, 5, X, y)

total rmse 13.05 std 1.5645


In [36]:
cb = cross_validation(CatBoostRegressor(n_estimators = 1000, max_depth = 9, verbose = 0), 1, 5, X, y)

total rmse 11.19 std 2.1913


In [37]:
model_train = pd.DataFrame()

In [43]:
model_train['RFR'] = cross_pred(rfr, X)
model_train['CB'] = cross_pred(cb, X)

In [52]:
model_ens_for = cross_validation(RandomForestRegressor(n_estimators = 3, max_depth = 20), 3, 10, model_train, y)

total rmse 7.387 std 0.998


In [55]:
model_ens_ridge = cross_validation(Ridge(), 3, 10, model_train, y)

total rmse 6.65 std 0.8119


In [64]:
pred_test = pd.DataFrame()
pred_test['RFR'] = cross_pred(cb, test)
pred_test['CB'] = cross_pred(rfr, test)

In [65]:
pred_test['emission'] = cross_pred(model_ens_ridge, pred_test)

In [68]:
sample_submission = pd.read_csv('sample_submission.csv')
base_submission = sample_submission.copy()
base_submission['emission'] = pred_test['emission']
base_submission.to_csv('stacking.csv', index = False)

In [69]:
base_submission

,ID_LAT_LON_YEAR_WEEK,emission
0,ID_-0.510_29.290_2022_00,3.217443
1,ID_-0.510_29.290_2022_01,4.023907
2,ID_-0.510_29.290_2022_02,4.158579
3,ID_-0.510_29.290_2022_03,3.990929
4,ID_-0.510_29.290_2022_04,4.195938
...,...,...
24348,ID_-3.299_30.301_2022_44,27.567506
24349,ID_-3.299_30.301_2022_45,29.099752
24350,ID_-3.299_30.301_2022_46,29.715700
24351,ID_-3.299_30.301_2022_47,29.854348
